In [204]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import csv
import xml.etree.ElementTree as ET
import copy
import random
import uuid

csv_folder_path = "../Interview CSV"

script_path = "../Script/test script.txt"
script_csv_output_path = "../Script/script_export.csv"

final_subtitle_file = "../Script/script.srt"

xml_folder = "../Interview XML/"
exported_xml_project_name= 'testy'

final_clip_list = []
id_mapping = {}
# global sequence_start
# global sequence_end

sequence_start=0
sequence_end=0
xml_file_assigned=[]

<h3> Helper functions </h3>

In [205]:
def timecode_to_frames(text):
    if len(text.split(":"))==2:
        frames=(int(text.split(":")[0])*60+int(text.split(":")[1]))*24
    elif len(text.split(":"))==3:
        frames=(int(text.split(":")[0])*3600+int(text.split(":")[1])*60+int(text.split(":")[2]))*24
    elif len(text.split(":"))==4:
        frames=(int(text.split(":")[0])*3600+int(text.split(":")[1])*60+int(text.split(":")[2]))*24+int(text.split(":")[3])
    else:
        print(text+"timecode parse error")
    return frames

timecode_to_frames("00:31:23:02")  #test

45194

In [206]:

def frames_to_timecode(frames):
    frames=int(frames)
    hours = frames // (3600*24)
    remaining_frames = frames % (3600*24)
    minutes=remaining_frames // (60*24)
    remaining_frames=remaining_frames % (60*24)
    seconds=remaining_frames // (24)
    remaining_frames =remaining_frames %(24)
    frames=remaining_frames



    timecode="{:02d}".format(hours)+":"+"{:02d}".format(minutes)+":"+"{:02d}".format(seconds)+":"+"{:02d}".format(frames)

    return timecode

#frames_to_timecode(45194)   #test

In [207]:
def clean_text(text):
    # Remove special characters from the text
    cleaned_text = re.sub(r"[^\w\s]", "", text)
    return cleaned_text

#clean_text("abcd$$")   #test

In [208]:
def match_line(line, narrator, df):
    cleaned_line = clean_text(line.lower())  
    matched_rows = []

    #df=df[df["NARRATOR"].str.lower()==narrator]
#    print(df)

    for index,row in df.iterrows():
        text = clean_text(str(row['TEXT']).lower())
        if cleaned_line in text:
            matched_rows.append(row) 
    matched_rows_df = pd.concat(matched_rows, axis=1).transpose() if matched_rows else pd.DataFrame()
    return matched_rows_df


##add test here

In [209]:
def get_character_indices(full_string, substring):
    full_list = list(full_string.lower())
    sub_list = list(substring.lower())
    length = len(sub_list)
    
    for i in range(len(full_list)):
        if full_list[i:i+length] == sub_list:
            return i, i+length
    return None, None

get_character_indices("abc def iopoi","def")   #test

(4, 7)

<h3> Convert Script to coded CSV </h3>

In [210]:

# The match_script_to_csv() function performs a matching operation between a script file and a
#  folder containing data from multiple CSV files
# in order to generate a csv version of the script with links to files and timecodes

def match_script_to_csv(script_path):

    combined_df = pd.DataFrame()

    for filename in os.listdir(csv_folder_path):
        if filename.endswith("csv"):
            file_path = os.path.join(csv_folder_path, filename)
            df = pd.read_csv(file_path)
            df["NARRATOR"]=(filename.split(" ")[1])
            df['FILEPATH'] = file_path
            combined_df = pd.concat([combined_df, df], ignore_index=True)

    matched_rows = []

    with open(script_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    narrator = None
    for line in lines:
        line=line.strip()
        if line=="":
            pass
        elif line[:3]=="###":
            narrator=line.strip()[3:]
        else:
            matched_rows_df = match_line(line, narrator, combined_df)
            cleaned_line = clean_text(line)
            if not matched_rows_df.empty:
                for index,row in matched_rows_df.iterrows():
                    print(row)
                    time_range = [s.strip() for s in row['TIME'].split("-")]
                    start_time = timecode_to_frames(time_range[0])
                    end_time = timecode_to_frames(time_range[1])
                    total_duration = end_time - start_time

                    text = clean_text(str(row['TEXT']).lower())

                    ##if the row matches completely, the new and original start/end times should match

                    print(text,cleaned_line)

                    start_index, end_index = get_character_indices(text, cleaned_line)
                    total_characters = len(list(text))

                    print(start_time,total_duration,total_characters,start_index)

                    new_start_time = start_time + total_duration/total_characters*start_index
                    new_end_time = start_time + total_duration/total_characters*end_index

                    new_row = {
                        'Text': line,
                        'Narrator': narrator,
                        'Timecode Range': frames_to_timecode(new_start_time) +" - " + frames_to_timecode(new_end_time),
                        'FilePath': row['FILEPATH']
                        }
            else:
                ##if no match is found, the line is added anyway but with no timecode or filepath
                new_row = {
                        'Text': line,
                        'Narrator': narrator,
                        'Timecode Range': None,
                        'FilePath': None
                    }
            matched_rows.append(new_row)

    matched_df = pd.DataFrame.from_records(matched_rows)
    return matched_df

# match_script_to_csv(script_path).to_csv(script_csv_output_path) #test

<h3> Generate subtitles from coded script csv </h3>

In [211]:
def convertTimecodeToSubFormat(timecode):
    timecodeSplit=timecode.split(":")
    timecodeSplit[3]=str(int(int(timecodeSplit[3])/24*1000))
    timecodeNew = ":".join(timecodeSplit[:3])+","+timecodeSplit[3]
    return timecodeNew

#convertTimecodeToSubFormat("01:02:03:04")   #test

In [212]:
    

# the generate_subtitles() function reads a script CSV file, performs time calculations
# and formatting, and writes the generated subtitles to a final subtitle file in the SRT format. if no time is found
# it adds 5 seconds to the last time.

def generate_subtitles(script_csv_file,final_subtitle_file):

    with open(final_subtitle_file, 'w') as srt_file:
        script_csv=pd.read_csv(script_csv_file)
        counter = 1
        last_time = 0
        for index,row in script_csv.iterrows():
            srt_file.write(str(counter) + '\n')
            
            new_start_frames=last_time

            if not pd.isna(row["Timecode Range"]):  # If Timecode Range is not empty

                start_time, end_time = [s.strip() for s in row["Timecode Range"].split('-')]
                start_frames=timecode_to_frames(start_time)
                end_frames=timecode_to_frames(end_time)

                new_end_frames= last_time+ end_frames - start_frames
                last_time=new_end_frames

            else:
                new_end_frames = new_start_frames+120
                last_time=new_end_frames

            srt_file.write(convertTimecodeToSubFormat(frames_to_timecode(new_start_frames)) +' --> ' + convertTimecodeToSubFormat(frames_to_timecode(new_end_frames)) + '\n')
            
            srt_file.write(clean_text(row["Text"]) + '\n\n')  # Cleaned text
            counter += 1

generate_subtitles(script_csv_output_path,final_subtitle_file)

Converts Time to Frames 

In [213]:
def convert_time_to_frames(time, rate):
    # print('Time', time, 'Rate', rate)
    parts = time.split(":")
    hours = int(parts[0])
    minutes = int(parts[1])
    seconds = int(parts[2])
    frames = int(parts[3])
    
    # Calculate the total duration in frames
    total_frames = (
        hours * 3600 * rate +  # Convert hours to frames
        minutes * 60 * rate +  # Convert minutes to frames
        seconds * rate +  # Convert seconds to frames
        frames
    )
    
    return total_frames


print(convert_time_to_frames('00:13:33:00', 24))

19512


Generate New ids for the each Clip item in seperate files

In [214]:
def generate_unique_id():
    unique_id = f"clipitem-{uuid.uuid4().hex}"
    return unique_id

generate_unique_id()

'clipitem-4eb8a0ce6ad44e898aae7ccef989d0c3'

Update the id of clips in specific XML file

In [215]:
def update_xml_ids(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    print('Assigning unique IDs to clipitem elements...')

    # Generate unique IDs for clipitem elements
      # Store the mapping of old IDs to new unique IDs
    for clip_item in root.findall('.//clipitem'):  # Adjust the XPath expression based on the actual location of clipitem elements
        current_id = clip_item.attrib['id']
        unique_id = generate_unique_id()
        id_mapping[current_id] = unique_id
        clip_item.attrib['id'] = unique_id

    
        # Update the references in the links section
    for link in root.findall('.//link'):
        linkclipref = link.find('linkclipref')
        if linkclipref is not None and linkclipref.text in id_mapping:
            linkclipref.text = id_mapping[linkclipref.text]
    print(id_mapping)

    # Save the modified XML file
    tree.write(xml_file, encoding="utf-8")

# update_xml_ids(xml_file)

Extract time in  start and end time format

In [216]:
def extract_timecode(timecode_range):
    start_time, end_time = timecode_range.split(" - ")
    start_time = start_time.strip().replace(" ", "")
    end_time = end_time.strip().replace(" ", "")
    print('Start Time',start_time,'End Time', end_time)
    return start_time, end_time

extract_timecode("00:00:00:00 - 00:23:00:00")

Start Time 00:00:00:00 End Time 00:23:00:00


('00:00:00:00', '00:23:00:00')

Convert frame to tick number

In [217]:
def frame_to_ticks(frame_number, frame_rate):
    total_ticks_in_a_second = 254016000000
    tick_value = int((frame_number * total_ticks_in_a_second) / frame_rate)
    return tick_value

frame_to_ticks(24, 24)

254016000000

This function goes through xml file and extract the clips that match with the start and end time using frame comparison

In [218]:

def extract_sequence_info(xml_file, start_time, end_time):
    global sequence_start, sequence_end
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extract sequence information
    sequence = root.find('sequence')
    sequence_info = {
        'duration': int(sequence.find('duration').text),
        'rate': {
            'timebase': int(sequence.find('rate/timebase').text),
            'ntsc': sequence.find('rate/ntsc').text == 'TRUE'
        }
    }
    sequence_rate = sequence_info['rate']['timebase']

    # Convert start and end times to frames
    start_frame = convert_time_to_frames(start_time, sequence_rate)
    end_frame = convert_time_to_frames(end_time, sequence_rate)
    proTickIn=frame_to_ticks(start_frame,sequence_rate)
    proTickOut=frame_to_ticks(end_frame,sequence_rate)
    
    print('Start and End Frame',start_frame ,end_frame)
    print(sequence_rate,'sequence_rate')

    # Extract video clip information
    video_clips = []
    for clip_item in root.findall('.//video//clipitem'):  # Only consider clip items within the <video> tag
        
        clip_info = {
            'id': clip_item.attrib['id'],  # Get the clip ID
            'name': clip_item.find('name').text,
            'duration': int(clip_item.find('duration').text),
            'rate': {
                'timebase': int(clip_item.find('rate/timebase').text),
                'ntsc': clip_item.find('rate/ntsc').text == 'TRUE'
            },
            'in': int(clip_item.find('in').text),
            'out': int(clip_item.find('out').text),
            'start': int(clip_item.find('start').text),
            'end': int(clip_item.find('end').text),
            'links': [],  # Initialize an empty list to store links,
            'video_clip_element': None,  # Store the clip item element for later use
            'linked_audio_clip_elements_list': []  # Initialize an empty list to store linked clip items
        }

        if clip_info['in'] <= start_frame <= clip_info['out']:
            sequence_start=sequence_end+10
            sequence_end=sequence_start+(end_frame-start_frame)

        links = clip_item.findall('link')
        for link in links:
            link_info = {
                'linkclipref': link.find('linkclipref').text,
                'mediatype': link.find('mediatype').text,
                'trackindex': int(link.find('trackindex').text),
                'clipindex': int(link.find('clipindex').text)
            }
            if link.find('groupindex') is not None:
                link_info['groupindex'] = int(link.find('groupindex').text)
            clip_info['links'].append(link_info)
            if link.find('mediatype').text == 'audio':
                audio_clip_items = root.findall('.//audio//clipitem')
                for audio_clip_item in audio_clip_items:
                    if audio_clip_item.attrib['id'] == link.find('linkclipref').text:
                        audio_clip_item.find('in').text = str(start_frame)  
                        audio_clip_item.find('out').text = str(end_frame)
                        audio_clip_item.find('start').text = str(sequence_start)  
                        audio_clip_item.find('end').text = str(sequence_end)
                        audio_clip_item.find('pproTicksIn').text = str(proTickIn)  
                        audio_clip_item.find('pproTicksOut').text = str(proTickOut)
                        clip_info['linked_audio_clip_elements_list'].append([audio_clip_item,int(link.find('trackindex').text)])

        # Check if the clip's in or out frame falls within the given start and end frames
        print('Clip Frame Inside file Found: ',clip_info['in'] <= start_frame <= clip_info['out'])
        print('Start - Matching - End')
        print(clip_info['in'],start_frame,clip_info['out'])
        if clip_info['in'] <= start_frame <= clip_info['out']:
            clip_item.find('in').text = str(start_frame)  
            clip_item.find('out').text = str(end_frame)
            clip_item.find('start').text = str(sequence_start)  
            clip_item.find('end').text = str(sequence_end)
            clip_info['video_clip_element']=clip_item
            video_clips.append(clip_info)
            print('Next Sequence Start',sequence_start)
            print('Next Sequence End',sequence_end)
            
        
    # print(clip_info, start_frame, end_frame)

    # Create result dictionary
    result = {
        'sequence_info': sequence_info,
        'video_clips': video_clips
    }
    # print('Result',result)

    if not video_clips:  # Check if video_clips list is empty
        return None
    
    return result


These functions go through all the xml files and call the  extract clips function

In [219]:
def process_csv_file(csv_file):
    xml_folder = "../Interview XML/"
    with open(csv_file, "r") as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            timecode_range = row["Timecode Range"]
            if timecode_range:
                start_time, end_time = extract_timecode(timecode_range)
                narrator_name = row["Narrator"]
                print('Narrator Name:' ,narrator_name)
                process_xml_files(xml_folder, start_time, end_time, narrator_name)


def process_xml_files(xml_folder, start_time, end_time, narrator_name):
    global xml_file_assigned,final_clip_list
    for filename in os.listdir(xml_folder):
        if filename.endswith(".xml") and narrator_name.lower() in filename.lower():
            # print(filename,narrator_name)
            xml_file = os.path.join(xml_folder, filename)
            if xml_file not in xml_file_assigned:
                update_xml_ids(xml_file)
                xml_file_assigned.append(xml_file)
            matched_clips = extract_sequence_info(xml_file, start_time, end_time)
            final_clip_list.append(matched_clips)

OLD Create XMl

In [220]:
def old_create_xml(xml_name,xml_json_data):
    
    # Create the root element
    root = ET.Element("xmeml", version="4")
    
    # Create the sequence element with attributes
    sequence = ET.SubElement(root, "sequence", id="sequence-2", TL_SQAudioVisibleBase="0", TL_SQVideoVisibleBase="0",
                             TL_SQVisibleBaseTime="0", TL_SQAVDividerPosition="0.5", TL_SQHideShyTracks="0",
                             TL_SQHeaderWidth="236", TL_SQDataTrackViewControlState="0",
                             Monitor_ProgramZoomOut="340011984312000", Monitor_ProgramZoomIn="0",
                             TL_SQTimePerPixel="1.6034289012958367", MZ_EditLine="333083126376000",
                             MZ_Sequence_PreviewFrameSizeHeight="1080", MZ_Sequence_PreviewFrameSizeWidth="1920",
                             MZ_Sequence_AudioTimeDisplayFormat="200", MZ_Sequence_PreviewUseMaxRenderQuality="false",
                             MZ_Sequence_PreviewUseMaxBitDepth="false", MZ_Sequence_VideoTimeDisplayFormat="110",
                             MZ_WorkOutPoint="15235011792000", MZ_WorkInPoint="0", MZ_ZeroPoint="0", explodedTracks="true")
    
    # Add the uuid element
    uuid = ET.SubElement(sequence, "uuid")
    uuid.text = "50e61931-251f-4069-8193-a3fbad7f93ff"
    
    # Add the duration element
    duration = ET.SubElement(sequence, "duration")
    duration.text = "31533"
    
    # Add the rate element with nested timebase and ntsc elements
    rate = ET.SubElement(sequence, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    
    # Add the name element
    name_element = ET.SubElement(sequence, "name")
    name_element.text = xml_name
    
    # Add the media element with nested video and audio elements
    media = ET.SubElement(sequence, "media")
    video = ET.SubElement(media, "video")

    # Add the format element with nested samplecharacteristics element
    format_ = ET.SubElement(video, "format")
    samplecharacteristics = ET.SubElement(format_, "samplecharacteristics")
    
    # Add the rate element with nested timebase and ntsc elements inside samplecharacteristics
    rate = ET.SubElement(samplecharacteristics, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    
    # Add the codec element with nested name and appspecificdata elements
    codec = ET.SubElement(samplecharacteristics, "codec")
    name = ET.SubElement(codec, "name")
    name.text = "Apple ProRes 422"
    appspecificdata = ET.SubElement(codec, "appspecificdata")
    
    # Add the appname, appmanufacturer, and appversion elements inside appspecificdata
    appname = ET.SubElement(appspecificdata, "appname")
    appname.text = "Final Cut Pro"
    appmanufacturer = ET.SubElement(appspecificdata, "appmanufacturer")
    appmanufacturer.text = "Apple Inc."
    appversion = ET.SubElement(appspecificdata, "appversion")
    appversion.text = "7.0"
    
    # Add the data element with nested qtcodec element inside appspecificdata
    data = ET.SubElement(appspecificdata, "data")
    qtcodec = ET.SubElement(data, "qtcodec")
    codecname = ET.SubElement(qtcodec, "codecname")
    codecname.text = "Apple ProRes 422"
    codectypename = ET.SubElement(qtcodec, "codectypename")
    codectypename.text = "Apple ProRes 422"
    codectypecode = ET.SubElement(qtcodec, "codectypecode")
    codectypecode.text = "apcn"
    codecvendorcode = ET.SubElement(qtcodec, "codecvendorcode")
    codecvendorcode.text = "appl"
    spatialquality = ET.SubElement(qtcodec, "spatialquality")
    spatialquality.text = "1024"
    temporalquality = ET.SubElement(qtcodec, "temporalquality")
    temporalquality.text = "0"
    keyframerate = ET.SubElement(qtcodec, "keyframerate")
    keyframerate.text = "0"
    datarate = ET.SubElement(qtcodec, "datarate")
    datarate.text = "0"
    
    # Add the width, height, anamorphic, pixelaspectratio, fielddominance, and colordepth elements inside samplecharacteristics
    width = ET.SubElement(samplecharacteristics, "width")
    width.text = "1920"
    height = ET.SubElement(samplecharacteristics, "height")
    height.text = "1080"
    anamorphic = ET.SubElement(samplecharacteristics, "anamorphic")
    anamorphic.text = "FALSE"
    pixelaspectratio = ET.SubElement(samplecharacteristics, "pixelaspectratio")
    pixelaspectratio.text = "square"
    fielddominance = ET.SubElement(samplecharacteristics, "fielddominance")
    fielddominance.text = "none"
    colordepth = ET.SubElement(samplecharacteristics, "colordepth")
    colordepth.text = "24"

    # Get the track elements
    video_tracks = [1]
    audio_tracks = []
    for clip in xml_json_data:

        audio_track_indexes = [link["trackindex"] for link in clip["video_clips"][0]["links"] if link["mediatype"] == "audio"]
        # print(audio_track_indexes)
        for audio_track_index in audio_track_indexes:
            if audio_track_index not in audio_tracks:
                audio_tracks.append(audio_track_index)

    # Create video track elements and append video clips
    for video_track_index in video_tracks:
        video_track = ET.SubElement(video, "track", TL_SQTrackShy="0", TL_SQTrackExpandedHeight="25",
                                    TL_SQTrackExpanded="0", MZ_TrackTargeted="0")
        # video_track.set("trackindex", str(video_track_index))

        for clip in xml_json_data:
            video_clip = clip["video_clips"][0]["video_clip_element"]
            video_track.append(copy.deepcopy(video_clip))

   
    

    
    # Add the audio element inside media
    audio = ET.SubElement(media, "audio")

    # constant stuff

    # Create subelements and append them to the audio element
    num_output_channels = ET.SubElement(audio, 'numOutputChannels')
    num_output_channels.text = '2'

    format_element = ET.SubElement(audio, 'format')
    sample_characteristics = ET.SubElement(format_element, 'samplecharacteristics')
    depth = ET.SubElement(sample_characteristics, 'depth')
    depth.text = '16'
    sample_rate = ET.SubElement(sample_characteristics, 'samplerate')
    sample_rate.text = '48000'

    outputs = ET.SubElement(audio, 'outputs')

    group_1 = ET.SubElement(outputs, 'group')
    index_1 = ET.SubElement(group_1, 'index')
    index_1.text = '1'
    num_channels_1 = ET.SubElement(group_1, 'numchannels')
    num_channels_1.text = '1'
    downmix_1 = ET.SubElement(group_1, 'downmix')
    downmix_1.text = '0'
    channel_1 = ET.SubElement(group_1, 'channel')
    channel_index_1 = ET.SubElement(channel_1, 'index')
    channel_index_1.text = '1'

    group_2 = ET.SubElement(outputs, 'group')
    index_2 = ET.SubElement(group_2, 'index')
    index_2.text = '2'
    num_channels_2 = ET.SubElement(group_2, 'numchannels')
    num_channels_2.text = '1'
    downmix_2 = ET.SubElement(group_2, 'downmix')
    downmix_2.text = '0'
    channel_2 = ET.SubElement(group_2, 'channel')
    channel_index_2 = ET.SubElement(channel_2, 'index')
    channel_index_2.text = '2'

     # Create audio track elements and append audio clips
    for audio_track_index in audio_tracks:
        audio_track = ET.SubElement(audio, "track", TL_SQTrackAudioKeyframeStyle="0", TL_SQTrackShy="0",
                                    TL_SQTrackExpandedHeight="25", TL_SQTrackExpanded="0",
                                    MZ_TrackTargeted="1", PannerCurrentValue="0.5", PannerIsInverted="true",
                                    PannerStartKeyframe="-91445760000000000,0.5,0,0,0,0,0,0", PannerName="Balance",
                                    currentExplodedTrackIndex=f"{audio_track_index-1}", totalExplodedTrackCount="2",
                                    premiereTrackType="Stereo")
        # audio_track.set("trackindex", str(audio_track_index))

        for clip in xml_json_data:
            audio_clip_elements = clip["video_clips"][0]["linked_audio_clip_elements_list"]
            
            for audio_clip in audio_clip_elements:
                if audio_track_index == audio_clip[1]:
                    audio_track.append(copy.deepcopy(audio_clip[0]))
    
    # Add the timecode element with nested rate, string, frame, and displayformat elements
    timecode = ET.SubElement(sequence, "timecode")
    rate = ET.SubElement(timecode, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    string = ET.SubElement(timecode, "string")
    string.text = "00:00:00:00"
    frame = ET.SubElement(timecode, "frame")
    frame.text = "0"
    displayformat = ET.SubElement(timecode, "displayformat")
    displayformat.text = "NDF"
    
    # Add the labels element with nested label2 element
    labels = ET.SubElement(sequence, "labels")
    label2 = ET.SubElement(labels, "label2")
    label2.text = "Forest"
    


    
    # Create the ElementTree object with the root element
    tree = ET.ElementTree(root)
    
    # Generate a random Idd
    idd = str(random.randint(1, 1000))
    
    # Save the XML to a file
    filename = f"../xml exports/{xml_name.replace(' ', '_')}-{idd}.xml"
    tree.write(filename, encoding="utf-8", xml_declaration=True)
    print(f"XML saved to {filename}")

<h3> Create XML </h3>

In [221]:

def create_xml(xml_name):
    
    # Create the root element
    root = ET.Element("xmeml", version="4")
    
    # Create the sequence element with attributes
    sequence = ET.SubElement(root, "sequence", id="sequence-2", TL_SQAudioVisibleBase="0", TL_SQVideoVisibleBase="0",
                             TL_SQVisibleBaseTime="0", TL_SQAVDividerPosition="0.5", TL_SQHideShyTracks="0",
                             TL_SQHeaderWidth="236", TL_SQDataTrackViewControlState="0",
                             Monitor_ProgramZoomOut="340011984312000", Monitor_ProgramZoomIn="0",
                             TL_SQTimePerPixel="1.6034289012958367", MZ_EditLine="333083126376000",
                             MZ_Sequence_PreviewFrameSizeHeight="1080", MZ_Sequence_PreviewFrameSizeWidth="1920",
                             MZ_Sequence_AudioTimeDisplayFormat="200", MZ_Sequence_PreviewUseMaxRenderQuality="false",
                             MZ_Sequence_PreviewUseMaxBitDepth="false", MZ_Sequence_VideoTimeDisplayFormat="110",
                             MZ_WorkOutPoint="15235011792000", MZ_WorkInPoint="0", MZ_ZeroPoint="0", explodedTracks="true")
    
    # Add the uuid element
    uuid = ET.SubElement(sequence, "uuid")
    uuid.text = "50e61931-251f-4069-8193-a3fbad7f93ff"
    
    # Add the duration element
    duration = ET.SubElement(sequence, "duration")
    duration.text = "31533"
    
    # Add the rate element with nested timebase and ntsc elements
    rate = ET.SubElement(sequence, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    
    # Add the name element
    name_element = ET.SubElement(sequence, "name")
    name_element.text = xml_name
    
    # Add the media element with nested video and audio elements
    media = ET.SubElement(sequence, "media")

    
 
    
    # Add the timecode element with nested rate, string, frame, and displayformat elements
    timecode = ET.SubElement(sequence, "timecode")
    rate = ET.SubElement(timecode, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    string = ET.SubElement(timecode, "string")
    string.text = "00:00:00:00"
    frame = ET.SubElement(timecode, "frame")
    frame.text = "0"
    displayformat = ET.SubElement(timecode, "displayformat")
    displayformat.text = "NDF"
    
    # Add the labels element with nested label2 element
    labels = ET.SubElement(sequence, "labels")
    label2 = ET.SubElement(labels, "label2")
    label2.text = "Forest"
    


    
    # Create the ElementTree object with the root element
    tree = ET.ElementTree(root)
    
    # Generate a random Idd
    idd = str(random.randint(1, 1000))
    
    # Save the XML to a file
    filename = f"{xml_name.replace(' ', '_')}-{idd}.xml"
    tree.write(f"../xml exports/{filename}", encoding="utf-8", xml_declaration=True)
    print(f"XML saved to {filename}")

    return filename 



Append Video to XML File

In [222]:
def append_video_to_xml(file_name, xml_json_data):
    print(f"Adding Video Tracks to {file_name}")
    # Parse the XML data
    tree = ET.parse(file_name)
    root = tree.getroot()

    # Find the media element
    media = root.find('.//media')
    print(tree)
    print(media)

    # Append the provided code inside the media element
    video = ET.SubElement(media, "video")

    # Add the format element with nested samplecharacteristics element
    format_ = ET.SubElement(video, "format")
    samplecharacteristics = ET.SubElement(format_, "samplecharacteristics")
    
    # Add the rate element with nested timebase and ntsc elements inside samplecharacteristics
    rate = ET.SubElement(samplecharacteristics, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    
    # Add the codec element with nested name and appspecificdata elements
    codec = ET.SubElement(samplecharacteristics, "codec")
    name = ET.SubElement(codec, "name")
    name.text = "Apple ProRes 422"
    appspecificdata = ET.SubElement(codec, "appspecificdata")
    
    # Add the appname, appmanufacturer, and appversion elements inside appspecificdata
    appname = ET.SubElement(appspecificdata, "appname")
    appname.text = "Final Cut Pro"
    appmanufacturer = ET.SubElement(appspecificdata, "appmanufacturer")
    appmanufacturer.text = "Apple Inc."
    appversion = ET.SubElement(appspecificdata, "appversion")
    appversion.text = "7.0"
    
    # Add the data element with nested qtcodec element inside appspecificdata
    data = ET.SubElement(appspecificdata, "data")
    qtcodec = ET.SubElement(data, "qtcodec")
    codecname = ET.SubElement(qtcodec, "codecname")
    codecname.text = "Apple ProRes 422"
    codectypename = ET.SubElement(qtcodec, "codectypename")
    codectypename.text = "Apple ProRes 422"
    codectypecode = ET.SubElement(qtcodec, "codectypecode")
    codectypecode.text = "apcn"
    codecvendorcode = ET.SubElement(qtcodec, "codecvendorcode")
    codecvendorcode.text = "appl"
    spatialquality = ET.SubElement(qtcodec, "spatialquality")
    spatialquality.text = "1024"
    temporalquality = ET.SubElement(qtcodec, "temporalquality")
    temporalquality.text = "0"
    keyframerate = ET.SubElement(qtcodec, "keyframerate")
    keyframerate.text = "0"
    datarate = ET.SubElement(qtcodec, "datarate")
    datarate.text = "0"
    
    # Add the width, height, anamorphic, pixelaspectratio, fielddominance, and colordepth elements inside samplecharacteristics
    width = ET.SubElement(samplecharacteristics, "width")
    width.text = "1920"
    height = ET.SubElement(samplecharacteristics, "height")
    height.text = "1080"
    anamorphic = ET.SubElement(samplecharacteristics, "anamorphic")
    anamorphic.text = "FALSE"
    pixelaspectratio = ET.SubElement(samplecharacteristics, "pixelaspectratio")
    pixelaspectratio.text = "square"
    fielddominance = ET.SubElement(samplecharacteristics, "fielddominance")
    fielddominance.text = "none"
    colordepth = ET.SubElement(samplecharacteristics, "colordepth")
    colordepth.text = "24"

    # Get the track elements
    video_tracks = [1]
    
    

    # Create video track elements and append video clips
    for video_track_index in video_tracks:
        video_track = ET.SubElement(video, "track", TL_SQTrackShy="0", TL_SQTrackExpandedHeight="25",
                                    TL_SQTrackExpanded="0", MZ_TrackTargeted="0")
        # video_track.set("trackindex", str(video_track_index))

        for clip in xml_json_data:
            video_clip = clip["video_clips"][0]["video_clip_element"]
            video_track.append(copy.deepcopy(video_clip))

    # Save the modified XML to a new file
    modified_tree = ET.ElementTree(root)
    modified_tree.write(file_name)    




Append Audio to XML File

In [223]:

def append_audio_to_xml(file_name, xml_json_data):
    print(f"Adding Audio Tracks to {file_name}")
    print(xml_json_data)
    # Parse the XML data
    audio_tracks = []
    for clip in xml_json_data:

        audio_track_indexes = [link["trackindex"] for link in clip["video_clips"][0]["links"] if link["mediatype"] == "audio"]
        # print(audio_track_indexes)
        for audio_track_index in audio_track_indexes:
            if audio_track_index not in audio_tracks:
                audio_tracks.append(audio_track_index)
    print(audio_tracks, "audio tracks")
    
    tree = ET.parse(file_name)
    root = tree.getroot()

    # Find the media element
    media = root.find('.//media')

    # Add the audio element inside media
    audio = ET.SubElement(media, "audio")

    # constant stuff

    # Create subelements and append them to the audio element
    num_output_channels = ET.SubElement(audio, 'numOutputChannels')
    num_output_channels.text = '2'

    format_element = ET.SubElement(audio, 'format')
    sample_characteristics = ET.SubElement(format_element, 'samplecharacteristics')
    depth = ET.SubElement(sample_characteristics, 'depth')
    depth.text = '16'
    sample_rate = ET.SubElement(sample_characteristics, 'samplerate')
    sample_rate.text = '48000'

    outputs = ET.SubElement(audio, 'outputs')

    group_1 = ET.SubElement(outputs, 'group')
    index_1 = ET.SubElement(group_1, 'index')
    index_1.text = '1'
    num_channels_1 = ET.SubElement(group_1, 'numchannels')
    num_channels_1.text = '1'
    downmix_1 = ET.SubElement(group_1, 'downmix')
    downmix_1.text = '0'
    channel_1 = ET.SubElement(group_1, 'channel')
    channel_index_1 = ET.SubElement(channel_1, 'index')
    channel_index_1.text = '1'

    group_2 = ET.SubElement(outputs, 'group')
    index_2 = ET.SubElement(group_2, 'index')
    index_2.text = '2'
    num_channels_2 = ET.SubElement(group_2, 'numchannels')
    num_channels_2.text = '1'
    downmix_2 = ET.SubElement(group_2, 'downmix')
    downmix_2.text = '0'
    channel_2 = ET.SubElement(group_2, 'channel')
    channel_index_2 = ET.SubElement(channel_2, 'index')
    channel_index_2.text = '2'

     # Create audio track elements and append audio clips
    for audio_track_index in audio_tracks:
        audio_track = ET.SubElement(audio, "track", TL_SQTrackAudioKeyframeStyle="0", TL_SQTrackShy="0",
                                    TL_SQTrackExpandedHeight="25", TL_SQTrackExpanded="0",
                                    MZ_TrackTargeted="1", PannerCurrentValue="0.5", PannerIsInverted="true",
                                    PannerStartKeyframe="-91445760000000000,0.5,0,0,0,0,0,0", PannerName="Balance",
                                    currentExplodedTrackIndex=f"{audio_track_index-1}", totalExplodedTrackCount="2",
                                    premiereTrackType="Stereo")
        # audio_track.set("trackindex", str(audio_track_index))
        print(audio_track_index, "audio track index")
        for clip in xml_json_data:
            audio_clip_elements = clip["video_clips"][0]["linked_audio_clip_elements_list"]
            
            for audio_clip in audio_clip_elements:
                if audio_track_index == audio_clip[1]:
                    audio_track.append(copy.deepcopy(audio_clip[0]))

   

    # Save the modified XML to a new file
    modified_tree = ET.ElementTree(root)
    modified_tree.write(file_name)


   

Extracts the given clips from the xml files and saves them in a seperate xml file

In [224]:
# the run_extraction() function serves as the main entry point for running the extraction process. 
# It calls functions to process a CSV file, filter the resulting list of clips, and create an XML file based on the filtered clips.

def run_extraction(script_csv_file,exported_xml_project_name):
    global final_clip_list    
    process_csv_file(script_csv_file)
    final_clip_list = [item for item in final_clip_list if item is not None]
    # print(final_clip_list)
    filename = create_xml(exported_xml_project_name)
    append_video_to_xml(f"../xml exports/{filename}", final_clip_list)
    append_audio_to_xml(f"../xml exports/{filename}", final_clip_list)
# print(final_clip_list)

Code Run Tests

In [225]:


csv_folder_path = "../Interview CSV"

script_path = "../Script/test script.txt"
script_csv_output_path = "../Script/script_export.csv"

final_subtitle_file = "../Script/script.srt"

exported_xml_project_name= 'testy'


#convert_txt_to_csv()

# match_script_to_csv(script_path)

# generate_subtitles(script_csv_output_path,final_subtitle_file)

run_extraction(script_csv_output_path,exported_xml_project_name)




Start Time 00:13:33:00 End Time 00:13:45:09
Narrator Name: Grandma
Assigning unique IDs to clipitem elements...
{'clipitem-bc429dff3450481cbd492f5adaf9f861': 'clipitem-d55914a2a5144a298b7a3eecd6613de0', 'clipitem-bb54742069b441e2a39317b58df54e13': 'clipitem-98f5458d46e94d158fbb8ceaffc384f4', 'clipitem-942638703f9e4531a3c4841aa2b602b7': 'clipitem-9084f2feaab34843bbdf6db1528f9f7c', 'clipitem-3dfde98999c344c9aab546ca9c0cbef9': 'clipitem-7f35a19f16134b28bcb78b466a5ececc', 'clipitem-c1b7ee83609b4f2f8149a9bad07a8898': 'clipitem-07e0af8fc6a64ebcb974bb579184f6cf', 'clipitem-9ed607cf1694469d964ec865ee96ea43': 'clipitem-eec3cd2fcfe84d2f87f3410e910e7416', 'clipitem-8522b99adaab426ca1d31ea18ac85ab8': 'clipitem-6d3077b13a7641e2a8d69fa564381984', 'clipitem-cee69a326c574856b0f50d1cf78caadf': 'clipitem-b5018d795a994e76afe7cd3b3d158e70', 'clipitem-797f9ede110d4c90b95e74ee359d8039': 'clipitem-8a4c6a945c184b219c5f3fbfa70424c7', 'clipitem-404ddba66da84f5b8d9d3c8fa947c659': 'clipitem-af64abac780e4f11aebe1f

Start and End Frame 21929 22011
24 sequence_rate
Clip Frame Inside file Found:  True
Start - Matching - End
4179 21929 52602
Next Sequence Start 1474
Next Sequence End 1556
Clip Frame Inside file Found:  True
Start - Matching - End
0 21929 39100
Next Sequence Start 1566
Next Sequence End 1648
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 2049
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 3573
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 4366
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 298
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 609
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 434
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 128
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 577
Clip Frame Inside file Found:  False
Start - Matching - End
0 21929 1654
Clip Frame Inside file Found:  False
Start - Matching 